In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
data_calls = pd.read_csv("/Users/macintosh/Desktop/VS code/calls.csv")
data_cease = pd.read_csv("/Users/macintosh/Desktop/VS code/cease.csv")

In [3]:
data_cease 

,unique_customer_identifier,cease_placed_date,cease_completed_date,reason_description,reason_description_insight
0,03b1c584533a86d067dd51bbca242db2b55b692f10d325...,2023-08-03,2023-09-04,Competitor Deals - No longer required,CompetitorDeals
1,97a7bdce317de91a32636e6675bbb2e5b25573308ef7bb...,2023-08-03,2023-09-04,Cease,VagueReason
2,c5049a1aedc36d7d7379c2c2144972b099521e6614cf8c...,2023-08-03,2023-09-05,Competitor Deals - No longer required,CompetitorDeals
3,cffa7eecb1708776f425a0f1e70598710b5e74a66d8fea...,2023-08-03,2023-08-21,Not Known,VagueReason
4,f450c8a916d400b204b0228fa23bf613ceae5727e08a68...,2023-08-03,2023-09-04,Competitor Deals - No longer required,CompetitorDeals
...,...,...,...,...,...
146358,0b8aa7af28ac58dd0ae47c240792c8fcf33bd113d31579...,2024-06-29,2024-07-12,Competitor Deals - Mobile and Broadband Deal,CompetitorDeals
146359,cf63c0bd7c1f0c22a3f5c156a1afe4396ae1c1877e4541...,2023-03-23,NaN,Not Known,VagueReason
146360,098814e650a83af558fbbbe82622bc27c02a5342541571...,2023-03-23,NaN,Not Known,VagueReason
146361,9de37d080f3c83582bbb9e342227521a61c696033670fb...,2023-10-04,NaN,Not Known,VagueReason


In [4]:
data_calls

,unique_customer_identifier,event_date,call_type,talk_time_seconds,hold_time_seconds
0,aae0258b41e6e88365d7d5ce648ea69d837602b4bb419e...,2023-02-22,Loyalty,627.0,235.0
1,15f9f6fc1872bbf6963a84de253d600e5d18d75d7784ce...,2023-03-16,Tech,267.0,293.0
2,c18d59888cb050a5694d1e613a277d79b4a3083bd1b813...,2023-02-22,Loyalty,689.0,0.0
3,1316da4b4282f98b572666413a71592352cc869c976212...,2023-03-16,Tech,3233.0,0.0
4,6f33a33df6e18900a60d495eb19f2c7b238a13ec5cd894...,2023-03-16,Tech,2.0,0.0
...,...,...,...,...,...
628432,56306182a47fc76ed76da2e100565d8d4ad4700748cc52...,2022-12-14,Tech,378.0,354.0
628433,dd3a7134b27b71f67d7e3a8a43a873035afe8c30dc5e1f...,2022-12-14,Tech,667.0,0.0
628434,70dd49147a12c1de2c1f531b964944a125bae9653bd3f4...,2023-05-24,Tech,544.0,0.0
628435,39fdde129407b5b7b4e4f056f02ba6d4c966e2f2682af2...,2023-04-10,Loyalty,442.0,97.0


My data cleaning approach is driven by the prediction goal: identifying customers likely to place a cease.
I clean data per table, preserve raw granularity, and only aggregate when it supports modelling.

In [5]:
data_cease.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146363 entries, 0 to 146362
Data columns (total 5 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   unique_customer_identifier  146363 non-null  object
 1   cease_placed_date           146363 non-null  object
 2   cease_completed_date        119146 non-null  object
 3   reason_description          146363 non-null  object
 4   reason_description_insight  146363 non-null  object
dtypes: object(5)
memory usage: 5.6+ MB


In [6]:
data_calls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 628437 entries, 0 to 628436
Data columns (total 5 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   unique_customer_identifier  628437 non-null  object 
 1   event_date                  628437 non-null  object 
 2   call_type                   610489 non-null  object 
 3   talk_time_seconds           628437 non-null  float64
 4   hold_time_seconds           628437 non-null  float64
dtypes: float64(2), object(3)
memory usage: 24.0+ MB


In [7]:
data_cease.describe()

,unique_customer_identifier,cease_placed_date,cease_completed_date,reason_description,reason_description_insight
count,146363,146363,119146,146363,146363
unique,130934,761,795,107,11
top,f912da887651950864f3a5415b207125832896bedf17bc...,2023-01-03,2023-10-25,Not Known,VagueReason
freq,19,436,1417,60010,76055


In [8]:
data_calls.describe()

,talk_time_seconds,hold_time_seconds
count,628437.000000,628437.000000
mean,680.093616,188.788232
std,662.157413,313.106665
min,0.000000,0.000000
25%,237.000000,0.000000
50%,504.000000,15.000000
75%,911.000000,271.000000
max,16017.000000,9003.000000


In [9]:
data_cease.isnull().sum()

unique_customer_identifier        0
cease_placed_date                 0
cease_completed_date          27217
reason_description                0
reason_description_insight        0
dtype: int64

In [10]:
grouped_calls = data_calls.groupby(['event_date', 'call_type']).size().reset_index(name='count')
print(grouped_calls)

      event_date         call_type  count
0     2022-08-01              CS&B    438
1     2022-08-01  Customer Finance    134
2     2022-08-01              FTTP     93
3     2022-08-01           Loyalty    197
4     2022-08-01  Order Management     12
...          ...               ...    ...
5067  2024-08-31             Other     10
5068  2024-08-31              Tech    107
5069  2024-09-01              CS&B     24
5070  2024-09-01             Other      1
5071  2024-09-01              Tech     10

[5072 rows x 3 columns]


In [11]:
data_calls.isnull().sum()

unique_customer_identifier        0
event_date                        0
call_type                     17948
talk_time_seconds                 0
hold_time_seconds                 0
dtype: int64

In [12]:
print(data_calls.duplicated().sum())
print(data_cease.duplicated().sum())

6486
350


In [13]:
clean_data_cease = data_cease.drop_duplicates()
clean_data_calls = data_calls.drop_duplicates()


In [14]:
print(data_cease.shape)
print(data_calls.shape)
print(clean_data_cease.shape)
print(clean_data_calls.shape)


(146363, 5)
(628437, 5)
(146013, 5)
(621951, 5)


In [15]:
print(clean_data_calls.duplicated().sum())
print(clean_data_cease.duplicated().sum())

0
0


After inspecting duplicates, I found a small number of exact row-level duplicates caused by data ingestion. I removed only those, which reduced the datasets by a few hundred to a few thousand rows while preserving all legitimate customer interactions.

Next, I will quantify missingness.

In [16]:
print(clean_data_cease.isna().sum())
print(clean_data_calls.isna().sum())

unique_customer_identifier        0
cease_placed_date                 0
cease_completed_date          26876
reason_description                0
reason_description_insight        0
dtype: int64
unique_customer_identifier        0
event_date                        0
call_type                     16806
talk_time_seconds                 0
hold_time_seconds                 0
dtype: int64


In [17]:
today = pd.to_datetime("today")
clean_data_cease['cease_placed_date'] = pd.to_datetime(clean_data_cease['cease_placed_date'])
clean_data_cease['cease_completed_date'] = pd.to_datetime(clean_data_cease['cease_completed_date'], errors='coerce')

# Duration in days
clean_data_cease['duration'] = np.where(
    clean_data_cease['cease_completed_date'].notnull(),
    (clean_data_cease['cease_completed_date'] - clean_data_cease['cease_placed_date']).dt.days,
    (today - clean_data_cease['cease_placed_date']).dt.days
)

# Event observed
clean_data_cease['Churn'] = clean_data_cease['cease_completed_date'].notnull().astype(int)

/var/folders/x2/xzn0478s38vfcsjrc5rzqfyr0000gn/T/ipykernel_54711/4273379803.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_data_cease['cease_placed_date'] = pd.to_datetime(clean_data_cease['cease_placed_date'])
/var/folders/x2/xzn0478s38vfcsjrc5rzqfyr0000gn/T/ipykernel_54711/4273379803.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_data_cease['cease_completed_date'] = pd.to_datetime(clean_data_cease['cease_completed_date'], errors='coerce')
/var/folders/x2/xzn0478s38vfcsjrc5rzqfyr0000g

In [18]:
clean_data_cease

,unique_customer_identifier,cease_placed_date,cease_completed_date,reason_description,reason_description_insight,duration,Churn
0,03b1c584533a86d067dd51bbca242db2b55b692f10d325...,2023-08-03,2023-09-04,Competitor Deals - No longer required,CompetitorDeals,32.0,1
1,97a7bdce317de91a32636e6675bbb2e5b25573308ef7bb...,2023-08-03,2023-09-04,Cease,VagueReason,32.0,1
2,c5049a1aedc36d7d7379c2c2144972b099521e6614cf8c...,2023-08-03,2023-09-05,Competitor Deals - No longer required,CompetitorDeals,33.0,1
3,cffa7eecb1708776f425a0f1e70598710b5e74a66d8fea...,2023-08-03,2023-08-21,Not Known,VagueReason,18.0,1
4,f450c8a916d400b204b0228fa23bf613ceae5727e08a68...,2023-08-03,2023-09-04,Competitor Deals - No longer required,CompetitorDeals,32.0,1
...,...,...,...,...,...,...,...
146354,a268c56536b409e6e4249cac2ff2950a297842b8424f89...,2024-06-29,2024-08-07,Competitor Deals - Better Broadband Deal,CompetitorDeals,39.0,1
146355,bce729de2730a950d3aa413bb9f8e138946c1915e6f6bb...,2024-06-29,2024-07-11,Competitor Deals - No longer required,CompetitorDeals,12.0,1
146356,3cc776e2d422c1102d9a57a40a5f47c9c6dff9dab0b659...,2024-06-29,NaT,Competitor Deals - No longer required,CompetitorDeals,596.0,0
146357,50e3fe7af5f1f517c8982870280048c69062d833324bc9...,2024-06-29,NaT,Not Known,VagueReason,596.0,0


Do not fill null cease dates because:
- Filling nulls with a fake date would confuse the model: it would think the customer ceased on that date, which is false.

In [19]:
# Drop rows with missing call type and zero talk time
clean_data_calls = clean_data_calls[~((clean_data_calls['call_type'].isna()) & (clean_data_calls['talk_time_seconds'] == 0))]

In [20]:
print(clean_data_calls.isna().sum())

unique_customer_identifier       0
event_date                       0
call_type                     3875
talk_time_seconds                0
hold_time_seconds                0
dtype: int64


Call type missing values drop to 3875 from 16806, eliminating unreal calls with no activity.

In [21]:
# Fill remaining missing call types with 'Unknown'
clean_data_calls['call_type'] = clean_data_calls['call_type'].fillna('Unknown')

In [22]:
# Check data again after cleaning
print(clean_data_cease.isna().sum())
print(clean_data_calls.isna().sum())

unique_customer_identifier        0
cease_placed_date                 0
cease_completed_date          26876
reason_description                0
reason_description_insight        0
duration                          0
Churn                             0
dtype: int64
unique_customer_identifier    0
event_date                    0
call_type                     0
talk_time_seconds             0
hold_time_seconds             0
dtype: int64


To merge the datasets, I will aggregate features in the call dataframe to avoid exploding the cease dataset into multiple rows per customer. This ensures that when the data is merged, it represents one row per call / customer

In [27]:
calls_agg = clean_data_calls.groupby('unique_customer_identifier').agg({
    'talk_time_seconds': ['mean', 'std'],    
    'hold_time_seconds': ['mean','max', 'std'],     
    'call_type': lambda x: x.mode()[0] if len(x) > 0 else 'Unknown',  
    'event_date': 'count'    
}).reset_index()

calls_agg.rename(columns={'event_date': 'num_calls'}, inplace=True)

# Flatten multi-level columns
calls_agg.columns = ['unique_customer_identifier', 'avg_talk_time_seconds', 'talk_time_seconds_std', 'avg_hold_time_seconds', 'max_hold_time', 'std_hold_time', 'call_type', 'num_calls']

In [28]:
# Combine datasets 
combined_data = pd.merge(clean_data_cease, calls_agg, on='unique_customer_identifier', how='left')

In [29]:
combined_data.isnull().sum()

unique_customer_identifier        0
cease_placed_date                 0
cease_completed_date          26876
reason_description                0
reason_description_insight        0
duration                          0
Churn                             0
avg_talk_time_seconds         40401
talk_time_seconds_std         67824
avg_hold_time_seconds         40401
max_hold_time                 40401
std_hold_time                 67824
call_type                     40401
num_calls                     40401
dtype: int64

In [30]:
# Fill NaNs for customers with no calls
combined_data['avg_talk_time_seconds'] = combined_data['avg_talk_time_seconds'].fillna(0)
combined_data['avg_hold_time_seconds'] = combined_data['avg_hold_time_seconds'].fillna(0)
combined_data['call_type'] = combined_data['call_type'].fillna('Unknown')
combined_data['num_calls'] = combined_data['num_calls'].fillna(0)


In [31]:
# Save cleaned data to new CSV file
combined_data.to_csv("/Users/macintosh/Desktop/VS code/combined_data.csv", index=False)